### Importación de Librerías:

In [1]:
import requests # permite hacer los request (POST/GET, etc.) a la API de InvertirOnline.com
import json
import csv
import numpy as np
import os 
import datetime 
import pandas as pd
import pickle
import time
import matplotlib.pyplot as plt

### Conexión a IOL y Bajada de Datos:

In [2]:
# Seleccionamos el período de tiempo del cual bajaremos los datos:

fecha_de_inicio_bajada = "2010-01-01"  # Desde esta fecha inclusive - FORMATO: Año-Mes-Día
fecha_de_final_bajada = "2022-11-30"   # Hasta esta fecha NO inclusive - FORMATO: Año-Mes-Día

In [3]:
# Creamos el token y pedimos acceso:

# ¡¡ TENEMOS QUE INGRESAR NUESTRA CONTRASEÑA !!

token = json.loads(requests.post(
    url = "https://api.invertironline.com/token",
    data = {
        "username":"_____________",
        "password":"_____________",
        "grant_type":"password"
            }           
).text)

In [4]:
# Pedimos los Requests a la API y guardamos los datos en la lista 'lista_de_jsons_con_datos_descargados':

acciones = {"ALUA", "BBAR", "BMA", "CEPU", "COME", "CRES", "EDN",
            "GGAL", "HARG", "LOMA", "MIRG", "PAMP", "TECO2", "TGNO4", "TGSU2",
            "TRAN", "TXAR", "AGRO", "BHIP", "BOLT", "BPAT", "CADO",
            "CELU", "FERR", "IRSA", "LEDE", "LONG", "MOLI", "SAMI",
            "SEMI", "BRIO", "CARC", "CECO2"}

lista_de_urls = []
lista_de_jsons_con_datos_descargados = []
lista_de_titulos_sin_ningun_dato = []

for accion in acciones:
    url = "https://api.invertironline.com/api/v2/bCBA/Titulos/{}/Cotizacion/seriehistorica/{}/{}/ajustada".format(accion, fecha_de_inicio_bajada, fecha_de_final_bajada)
    datos_en_formato_json = json.loads( requests.get(url, headers={"Authorization":"Bearer "+token["access_token"]}).text )

    # Anotamos los nombres de las acciones cuyos Requests no devuelven nada:
    if len(datos_en_formato_json) != 0:   
        datos_en_formato_json [0]["titulo"] = accion
    else:
        lista_de_titulos_sin_ningun_dato.append(accion)
    
    lista_de_urls.append(url)
    lista_de_jsons_con_datos_descargados.append(datos_en_formato_json)

In [5]:
# Estructuramos los datos en varios DataFrames (un DataFrame por cada acción) y luego los exportamos todos a un archivo .csv

lista_de_dataframes = []
for conjunto_de_datos_de_una_accion in lista_de_jsons_con_datos_descargados:
    
    # Creamos las siguientes listas :
    titulo=[]; cierre=[] ; apertura=[]; maximo=[] ;minimo=[]; fecha=[]; volumen=[]
    
    for j in range(len(conjunto_de_datos_de_una_accion)):
        titulo.append (conjunto_de_datos_de_una_accion[0]["titulo"])
        cierre.append (conjunto_de_datos_de_una_accion[j]["ultimoPrecio"]) 
        apertura.append (conjunto_de_datos_de_una_accion[j]["apertura"]) 
        maximo.append (conjunto_de_datos_de_una_accion[j]["maximo"]) 
        minimo.append (conjunto_de_datos_de_una_accion[j]["minimo"]) 
        fecha.append (conjunto_de_datos_de_una_accion[j]["fechaHora"]) 
        volumen.append (conjunto_de_datos_de_una_accion[j]["volumenNominal"])
    
    df = pd.DataFrame()
    df = pd.DataFrame({
                    "titulo":titulo,
                    "fecha":fecha,
                    "apertura":apertura,
                    "cierre":cierre,
                    "máximo":maximo,
                    "mínimo":minimo,
                    "volumen":volumen})
    
    # Ordenamos el DataFrame para que queden las fechas de lo más antiguo a lo más reciente:
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    
    # Guardamos el DataFrame con todos los datos de esa acción en la siguiente lista:
    lista_de_dataframes.append(df)
    

In [6]:
# Creamos un DataFrame que reúna los datos de todas las acciones:

df_completo = pd.concat(lista_de_dataframes, ignore_index=False)
df_completo = df_completo.reset_index(drop=True)

In [ ]:
# Mostramos en pantalla aquellas acciones de las que no se pudo obtener ningún dato:

if len(lista_de_titulos_sin_ningun_dato)!= 0:
    print ("Acciones de las que no se pudo obtener ningún dato:")
    for accion in lista_de_titulos_sin_ningun_dato:
        print("-", accion)       
        
# Mostramos en pantalla la cantidad de acciones que sí se descargaron datos:
print ("\nLa cantidad de acciones de las que sí se obtuvieron datos de la API es:",
       len(df_completo.titulo.unique()))

### Exportación:

In [8]:
# Creamos la carpeta donde guardaremos los datos:
if not os.path.exists('Datos_descargados'):
    os.makedirs('Datos_descargados')

# Exportamos a .csv
df_completo.to_csv("./Datos_descargados/Datos_Completos.csv",  index = False )